# **OUTDATED** - Look at `oas_calculation_polars`

In [ ]:
!pip install -r ../requirements.txt

In [ ]:
import pandas as pd
import QuantLib as ql
import numpy as np
import matplotlib.pyplot as plt 
from typing import Literal
import math
import warnings
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)

# Load Bond Returns data and Zero Rate Curve

## Bond Returns + Data Cleaning

For the definitions of the features, refer to the pdf in the `docs` folder

Variable Summary: 

* `cs`: credit spread computed as bond_yield in excess of duration-matched portfolio of US Treasuries yields
* `tmt`: time to maturity (in months, I guess)
* `ind_num_17`: Fama-French 17 Industry Classification (may be useful for value factor)
* `size_ig`, `size_jk`: dummy for (respectively) IG/HY bonds in the BAML/ICE index
* `bond_type`: US Corporate Convertible (CCOV), US Corporate Debentures (CDEB), US Corporate Medium Term Note (CMTN), US Corporate Medium Term Note Zero (CMTZ), or US Corporate Paper (CP)
* `R_FR`, `N_FR` and co.: rating as names and one-hot encoded, probably from different providers (SP may be S&P, FR Fitch Ratings, MR Moody's
* `INTEREST_FREQUENCY`: e.g. 13 for variable coupon, -1 for NA, 14 for bi-monthly, 15-16 undocumented
* `DATED_DATE`: date from which the bond interest accrues
* Additional Info on variables: FISD data dictionary 2012 document

Prices Variables

* `PRICE_EOM`: considers all trading days and takes the last
* `PRICE_LDM`: consider only last trading day of the month otherwise NaN
* `PRICE_L5M`: consider only last 5 trading days of the month otherwise NaN
* `T_SPREAD`: average trade-weighted bid-ask spread
* `T_YLD_PT`: average trade-weighted yield point
* `T_VOLUME`: volume traded during the month, par-value
* `T_DVOLUME`: volume traded during the month, dollar value
* `bondprc` is adjusted for MMN, `BONDPRC` is unadjusted

Other notes

* We may want to remove defaulted bonds (check if they were actually already removed)

In [ ]:
date_cols = [
    'date', 
    'MATURITY',
    'OFFERING_DATE', 
    'FIRST_INTEREST_DATE', 
    'LAST_INTEREST_DATE',
    'nextcoup',
    'DEFAULT_DATE',
    'REINSTATED_DATE',
    'DATED_DATE'
]

bond_data = pd.read_csv('../data/bond_data_final.csv', index_col=0, parse_dates=date_cols)

bond_data.iloc[:2,:10]

In [ ]:
# merge duration across the two dataframes
bond_data['duration'] = bond_data['DURATION_y'].fillna(bond_data['DURATION_x'])
bond_data.drop(['DURATION_x', 'DURATION_y'], axis=1, inplace=True)

In [ ]:
useless_cols = [
    'company_symbol', # we do not need it
    'TREASURY_MATURITY', # we do not need it
    'CONV', # convertible bonds (we already removed all convertibles) 
    'sic_code', # SIC Industry Code (we don't need it) 
    'mom6_1', # 6m momentum (we don't need it) 
    'ltrev48_12', # sum of bond returns over 48months (momentum) 
    'TMT', # time to maturity in years (we have it in months in tmt) and TMT has NaNs while tmt does not
    'DATE', # has NaNs while date does not
    'CUSIP', # has Nans while cusip does not
    'COUPON', # repeated in coupon and COUPON has NaNs while coupon does not
]

bond_data.drop(useless_cols, axis=1, inplace=True, errors='ignore')

In [ ]:
# fill all DATED_DATE, OFFERING_DATE, MATURITY, and DAY_COUNT_BASIS since they are all the same for each issue
cols_to_fill = ['DATED_DATE', 'OFFERING_DATE', 'DAY_COUNT_BASIS', 'MATURITY', 'NCOUPS']
grouped = bond_data.groupby('cusip')
def fill_dates(group): 
    group[cols_to_fill] = group[cols_to_fill].ffill().bfill()
    return group 

filled_data = grouped.apply(fill_dates, include_groups=False)
bond_data = filled_data.reset_index(0)

In [ ]:
# remove the CUSIPS for which there is no DATED_DATE, OFFERING_DATE and DAY COUNT BASIS? we should probably do that
new_bond_data = bond_data.dropna(subset=cols_to_fill)

removed_cusips = bond_data['cusip'].nunique() - new_bond_data['cusip'].nunique()
bond_data = new_bond_data

print(f'Removed {removed_cusips} CUSIPs from the DataFrame')

In [ ]:
required_cols = ['coupon', 'date'] + cols_to_fill

bond_data[required_cols].isna().sum()

Great, now we have all variables needed for the analysis!

# Load the historical zero curve

In [ ]:
yield_curve = pd.read_csv('../data/yield_panel_monthly_frequency_daily_maturity.csv', index_col=0)
yield_curve.index = pd.to_datetime(yield_curve.index)
yield_curve.drop('MAX_DATA_TTM', axis=1, inplace=True)  

In [ ]:
yield_curve.dtypes

In [ ]:
yield_curve = yield_curve.resample('ME').last()


In [ ]:
yield_curve.iloc[:10, :10]

# OAS Calculation with QuantLib

OAS is the spread that added to the zero rates in the pricing function returns the price of the bond. We use the Newton method to get a solution for the OAS. In our case, since we stripped bonds with optionality, the OAS is the Z-Spread

In [ ]:
mask = (bond_data['tmt'] / 12) < 30

bond_data[mask].shape

In [ ]:
bond_data[mask].shape[0] / bond_data.shape[0]

In [ ]:
bond_data = bond_data[mask]

In [ ]:
bond_data['bondprc'].isna().sum() / bond_data.shape[0]

In [ ]:
mask_bond = (bond_data['cusip'] == '00103YAE1') &(bond_data['date'] == '2002-08-31') 
example_bond = bond_data.loc[mask_bond].iloc[0] 

example_bond[:5]

In [ ]:
bond_data[bond_data['bondprc'] < 2]

In [ ]:
mask = bond_data['NCOUPS'] > 0
bond_data[mask]['cusip'].nunique() / bond_data['cusip'].nunique()

bond_data = bond_data[mask]

In [ ]:
example_bond.DAY_COUNT_BASIS

In [ ]:
8.54 / 4 /100 * 100

In [ ]:
bond_data.columns

In [ ]:
bond_data['DAY_COUNT_BASIS'].unique()

In [ ]:
def decompose_date(date: pd.Timestamp):
    """
    Returns day, month, year given a `pd.Timestamp`
    Parameters
    ----------
    date: pd.Timestamp

    Returns
    -------
    tuple[int, int, int]: day, month, year

    """
    return date.day, date.month, date.year

def get_day_count(bond: pd.Series): 
    day_count_convention = bond.DAY_COUNT_BASIS
    
    if day_count_convention == '30/360': 
        return ql.Thirty360(ql.Thirty360.USA)
    elif day_count_convention == 'ACT/360': 
        return ql.Actual360()
    elif day_count_convention == 'ACT/ACT': 
        return ql.ActualActual(ql.ActualActual.Bond)
    
    raise Exception(f'we did not implement day count {day_count_convention}')
    
def get_coupon_freq(bond: pd.Series): 
    coupon_freq = bond.NCOUPS
    if coupon_freq == 1: 
        return ql.Period(ql.Annual)
    elif coupon_freq == 2: 
        return ql.Period(ql.Semiannual)
    elif coupon_freq == 4: 
        return ql.Period(ql.Quarterly)
    elif coupon_freq == 12: 
        return ql.Period(ql.Monthly)
    
    raise Exception(f'we did not implement coupon freq {coupon_freq}')

In [ ]:

def get_zero_curve(date: ql.Date, calendar: ql.Calendar, maturity_freq: Literal['daily', 'monthly']) -> ql.ZeroCurve: 
    # get the zero rates for that specific date
    date_mask = pd.to_datetime(date.to_date())
    zero_rates = yield_curve.loc[date_mask]
    zero_rates: pd.Series
    
    # create the list of tenors based on the number of observations
    tenors = np.arange(0, len(zero_rates) + 1)
    
    # set the tenor unit and compounding frequency based on the type of data used
    if maturity_freq == 'daily':
        tenor_unit = ql.Days
        compounding_freq = ql.Daily
    elif maturity_freq == 'monthly': 
        tenor_unit = ql.Months
        compounding_freq = ql.Monthly
    else: 
        raise Exception(f'maturity not valid')
    
    # create the list of spot dates and rates
    #   (need to add a point for the evaluation date, hence the 0.)
    spot_dates = [date + ql.Period(tenor.item(), tenor_unit) for tenor in tenors] 
    spot_rates = [0.] + zero_rates.to_list()
    
    # set payment convention as specified in the paper (365 days)
    pmt_convention = ql.Actual365Fixed(ql.Actual365Fixed.Standard)
    
    # create the ZeroCurve and return it
    calendar = ql.UnitedStates(ql.UnitedStates.SOFR)
    spot_curve = ql.ZeroCurve(spot_dates, spot_rates, pmt_convention, calendar, ql.Linear(), ql.Compounded, ql.Continuous)
    
    return spot_curve

def debug_cashflows(bond: ql.FixedRateBond, bond_data: pd.Series, mkt_price: float, z_spread: float, impl_clean_price: float):
    """
    Debug cashflows given a bond and bond_data. 
    
    Function to debug the results of the OAS calcuations.
    
    Parameters
    ----------
    bond
    bond_data
    """
    cashflows = bond.cashflows()
    print('--- BOND SETUP & CALCS CHECKS ---')
    print(f'\tCalc Date = {bond_data.date}, \n\tOffering date = {bond_data.OFFERING_DATE}, Maturity = {bond_data.MATURITY}')
    
    # check for coupon_amt
    data_coupon_amt = bond_data.coupon * 100 / bond_data.NCOUPS / 100 # todo account for the coupon frequency
    bond_ql_coup_amt = np.round(cashflows[2].amount(), 2)
    print(f'\tCoupon Check: Data = {data_coupon_amt}, Model = {bond_ql_coup_amt}')
        
    # check that Accrued Interest
    data_accrued_interest = bond_data.COUPACC
    bond_ql_accr_interest = np.round(bond.dirtyPrice() - bond.cleanPrice(), 2)
    print(f'\tAccrued Interest Check: Data = {data_accrued_interest}, Model = {bond_ql_accr_interest}')
    
    print('\tCASHFLOWS SCHEDULE')
    for c in cashflows:
        print('\t%20s %12f' % (c.date(), c.amount()))
        
    coupons = [ql.as_coupon(c) for c in bond.cashflows()[:-1]]
    coupons_df = pd.DataFrame([(c.date().to_date(), c.rate(), c.accrualPeriod()) for c in coupons], columns=['date', 'rate', 'accrual_period'], index=range(1,len(coupons)+1))
    print(coupons_df)
    
    # checks for coupon dates
    bond_first_pmt_date = bond_data.FIRST_INTEREST_DATE.date()
    bond_last_pmt_date = bond_data.LAST_INTEREST_DATE.date()
    bond_ql_first_pmt_date = cashflows[0].date().to_date()
    bond_ql_last_pmt_date = cashflows[-3].date().to_date()
    
    first_delta = (bond_first_pmt_date - bond_ql_first_pmt_date).days
    last_delta = (bond_last_pmt_date - bond_ql_last_pmt_date).days
    
    print('\tChecks for Payment Dates')
    print(f'\t\tFirst pmt: Data = {bond_first_pmt_date}, Model = {bond_ql_first_pmt_date}, Delta = {first_delta}')
    print(f'\t\tLast pmt: Data = {bond_last_pmt_date}, Model = {bond_ql_last_pmt_date}, Delta = {last_delta}')
    
    
    delta_p = mkt_price - impl_clean_price
    delta_bps = delta_p / mkt_price * 100 * 100

    print(f'\tZ-SPREAD = {z_spread:.5f} ({z_spread * 100:.3f}%)')
    print(f'\tMkt Price = {mkt_price}, Implied Clean Price = {impl_clean_price:.5f}, Delta = {delta_p:.5f}, Delta (bps): {delta_bps:.2f}')
    
    print(f'DEBUG: {data_coupon_amt} {bond_ql_coup_amt}')
    assert math.isclose(data_coupon_amt, bond_ql_coup_amt, rel_tol=1e-2)
    if not math.isclose(data_accrued_interest, bond_ql_accr_interest): 
        warnings.warn('Accrued Interest is not correct')
    # assert math.isclose(data_accrued_interest, bond_ql_accr_interest) 
    assert bond_first_pmt_date == bond_ql_first_pmt_date # check the first payment date matches
    print('--- ALL CHECKS PASSED FOR BOND CALCULATIONS ---')
    print('--- CHECKS FOR Z-SPREAD CALCULATIONS ---')
    assert abs(first_delta) < 3
    assert abs(last_delta) < 3
    # delta p less than 1bp
    assert abs(delta_p) < 0.05
    print('--- ALL CHECKS PASSED FOR Z-SPREAD CALCULATION ---')
    
class ParameterNaNException(Exception):
    def __init__(self, varname: str):
        self.msg = f'Variable {varname} is NaN, and it is required.'
        super().__init__(self.msg)
        
def check_parameters(bond: pd.Series): 
    for varname in ['coupon', 'principal_amt']: 
        if np.isnan(bond[varname]): raise ParameterNaNException(varname)
    
    for varname in ['date', 'OFFERING_DATE', 'MATURITY', 'DATED_DATE']: 
        if pd.isnull(bond[varname]): raise ParameterNaNException(varname)
    
def compute_OAS(bond: pd.Series, debug: bool = False):
    # check that parameters are defined
    print(f'computing OAS for bond {bond.cusip} at {bond.date.date()}...', end='')
    if np.isnan(bond.bondprc): 
        print('No price data, skipping this row')
        return np.nan
    check_parameters(bond)
    
    calc_date = ql.Date(*decompose_date(bond.date))
    ql.Settings.instance().evaluationDate = calc_date
    
    # key data
    calendar = ql.UnitedStates(ql.UnitedStates.NYSE) # calendar to follow for calculations
    calendar = ql.NullCalendar()
    day_count_convention = get_day_count(bond) # the day count convention as specified in the bond
    day_count_convention = ql.ActualActual(ql.ActualActual.Bond) # the day count convention as specified in the bond
    
    # bond data
    issue_date = ql.Date(*decompose_date(bond.OFFERING_DATE))
    accruing_start_date = ql.Date(*decompose_date(bond.DATED_DATE)) # this is the date from which the bond starts accruing interest
    maturity_date = ql.Date(*decompose_date(bond.MATURITY))
    tenor = get_coupon_freq(bond)
    date_generation = ql.DateGeneration.Backward
    month_end = False
    face_value = bond.principal_amt
    face_value = 100
    coupon = bond.coupon / 100
    mkt_price = bond.bondprc
    first_pmt_date = ql.Date(*decompose_date(bond.FIRST_INTEREST_DATE))
    
    schedule = ql.Schedule(accruing_start_date, maturity_date, tenor, calendar, ql.Unadjusted, ql.Unadjusted, date_generation, month_end, first_pmt_date)
    
    settlement_days = 0
    
    # zero curve
    spot_curve = get_zero_curve(calc_date, calendar, 'daily')
    pricing_curve = ql.YieldTermStructureHandle(spot_curve)
    
    bond_ql = ql.FixedRateBond(
        settlement_days, 
        face_value, 
        schedule, 
        [coupon],
        day_count_convention
    )
    bond_ql.setPricingEngine(ql.DiscountingBondEngine(pricing_curve))
    
    # Z-spread calculation 
    z_spread = ql.BondFunctions.zSpread(
        bond_ql, 
        mkt_price,
        spot_curve,
        day_count_convention, 
        ql.Compounded,
        ql.Continuous, 
        calc_date,
        1.e-16,
        10_000_000,
        0.
    )
    
    def get_impl_clean_price(spread):
        spread1 = ql.SimpleQuote(spread)
        spread_handle1 = ql.QuoteHandle(spread1)
        ts_spreaded1 = ql.ZeroSpreadedTermStructure(pricing_curve,
                                                    spread_handle1,
                                                    ql.Compounded,
                                                    ql.Continuous)
        ts_spreaded_handle1 = ql.YieldTermStructureHandle(ts_spreaded1)
        fixed_rate_bond = ql.FixedRateBond(settlement_days,
                                        face_value,
                                        schedule,
                                        [coupon],
                                        day_count_convention)
        # Set Valuation engine
        bond_engine = ql.DiscountingBondEngine(ts_spreaded_handle1)
        fixed_rate_bond.setPricingEngine(bond_engine)
        value = fixed_rate_bond.cleanPrice()
        print(f'bond NPV: {fixed_rate_bond.NPV()}, clean: {fixed_rate_bond.cleanPrice()}')
        return value
    
    if debug: 
        impl_clean_price = get_impl_clean_price(z_spread)
        debug_cashflows(bond_ql, bond, mkt_price, z_spread, impl_clean_price)
        
    print(f' ...Z-spread is {z_spread}')

    return z_spread

In [ ]:
spot_crv = compute_OAS(example_bond)

In [ ]:
bond_data.sort_values(['date', 'cusip'], inplace=True)

In [ ]:
bond_data[bond_data['date'] == '2002-8-31']

In [ ]:
OAS = bond_data.apply(compute_OAS, axis=1)

In [ ]:
OAS.head()

In [ ]:
def plot_yc(date, curve: ql.ZeroCurve = None): 
    date_mask = date
    zero_rates = yield_curve.loc[date_mask]
    
    fig, ax = plt.subplots()
    
    dates, rates = zip(*curve.nodes())
    dates = [d.to_date() for d in dates]
    
    print(dates[:3])
    print(rates[:3])
    
    ax.plot(dates[1:], zero_rates, lw='5')
    ax.plot(dates, rates, c='red')
    
plot_yc('2012-04-30', spot_crv)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=809bdc60-6cc4-4659-aae8-7be15b203bd4' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>